In [2]:
import numpy as np
import math
import random
import warnings
from scipy.integrate import quad
from scipy.optimize import root_scalar
import matplotlib.pyplot as plt

In [1]:
def boxcar_smooth(data, window_size):
  if window_size <= 0:
    raise ValueError("Window size must be positive")
  kernel = np.ones(window_size) / window_size
  smoothed_data = np.convolve(data, kernel, mode='same')
  return smoothed_data

In [15]:
x_data = np.array([0, 1, 1.25, 1.5, 1.75, 2, 2.5, 3, 4])
y_data = np.array([10, 20, 25, 40, 10, 5, 10, 5, 10])
estradiol_level = np.interp(np.linspace(0,4,100)*24, x_data*24, y_data)

In [ ]:
tmat = np.load('DtoP_transition_matrix.npy')

array([[0.51740506, 0.21212121, 0.11888112, 0.15485075, 0.09053498],
       [0.02689873, 0.09090909, 0.05594406, 0.00746269, 0.03292181],
       [0.17563291, 0.39393939, 0.44055944, 0.16604478, 0.23045267],
       [0.22151899, 0.18181818, 0.26573427, 0.54664179, 0.21399177],
       [0.0585443 , 0.12121212, 0.11888112, 0.125     , 0.43209877]])

In [34]:
state_change_matrix = np.array([
    [1,0,0,0],  # 1.  New F grown
    [0,1,0,0],  # 2.  New H grown
    [0,0,1,0],  # 3.  New S grown
    [0,0,0,1],  # 4.  New M grown
    [-1,0,0,0], # 5.  Existing F pruned
    [0,-1,0,0], # 6.  Existing H pruned
    [0,0,-1,0], # 7.  Existing S pruned
    [0,0,0,-1], # 8.  Existing M pruned
    [1,-1,0,0], # 9.  H transitions to F
    [1,0,-1,0], # 10. S transitions to F
    [1,0,0,-1], # 11. M transitions to F
    [-1,1,0,0], # 12. F transitions to H
    [-1,0,1,0], # 13. F transitions to S
    [-1,0,0,1], # 14. F transitions to M
    [0,1,-1,0], # 15. S transitions to H
    [0,1,0,-1], # 16. M transitions to H
    [0,1,0,-1], # 17. H transitions to S
    [0,0,1,-1], # 18. M transitions to S
    [0,-1,0,1], # 19. H trantisions to M
    [0,0,-1,1]  # 20. S transitions to M
]).T
state_change_matrix

array([[ 1,  0,  0,  0, -1,  0,  0,  0,  1,  1,  1, -1, -1, -1,  0,  0,
         0,  0,  0,  0],
       [ 0,  1,  0,  0,  0, -1,  0,  0, -1,  0,  0,  1,  0,  0,  1,  1,
         1,  0, -1,  0],
       [ 0,  0,  1,  0,  0,  0, -1,  0,  0, -1,  0,  0,  1,  0, -1,  0,
         0,  1,  0, -1],
       [ 0,  0,  0,  1,  0,  0,  0, -1,  0,  0, -1,  0,  0,  1,  0, -1,
        -1, -1,  1,  1]])

In [35]:
def find_closest_timestamp(arr, t):
    ind = np.nanargmin(np.abs(arr - t))
    approx_t = arr[ind]
    return ind, approx_t

In [ ]:
# environmental function
def estradiol_lvl(t):
    # `t` must be aligned so that it always begins at the start
    # of diestrus stage.

    # Mod so that any number of hours falls into the 0 to 96 hour range
    t = t%(4*24)
    cycle_ind, _ = find_closest_timestamp(np.linspace(0,4,100)*24, t)

    # return the current estradiol level
    return estradiol_level[cycle_ind]

In [43]:
symbol_matrix = np.array([
    ['deltaM','gammaM2F','gammaM2H','gammaM2S','lambdaM'],
    ['deltaS','gammaS2F','gammaS2H','lambdaS','gammaS2M'],
    ['deltaH','gammaH2F','lambdaH','gammaH2S','gammaH2M'],
    ['deltaF','lambdaF','gammaF2H','gammaF2S','gammaF2M'],
    ['0','betaF','betaH','betaS','betaM']
])

# Load transition probability matrices
DtoP = np.load('DtoP_transition_matrix.npy')
PtoE = np.load('PtoE_transition_matrix.npy')
EtoM = np.load('EtoM_transition_matrix.npy')
MtoD = np.load('MtoD_transition_matrix.npy')

mats = [DtoP, PtoE, EtoM, MtoD]

transition_dict = {}
# Iterate through parameters
for i in range(5): # from
    for j in range(5): # to

        # Get the values across stages
        transvals_across_stages = [mat[i, j] for mat in mats]

        # Interpolate to bins
        transvals_at_bins = np.interp(
            np.linspace(0,4,100)*24,
            np.arange(4)*24,
            transvals_across_stages
        )

        # add to dict
        transition_dict[symbol_matrix[i,j]] = transvals_at_bins

In [ ]:
# Given an estradiol level, can you decode the current transition
# probabilities, or is it ambigious?




In [ ]:
# Homogeneous Poisson Process (hpp) inter-event time sampler
def hpp(intentemp):
    total_intensity = sum(intentemp)
    # Avoid division by zero (should be handled by caller)
    return - (1 / total_intensity) * math.log(1 - random.random())

# Nonhomogeneous Poisson Process (nhpp) inter-event time sampler using inverse transform
def nhpp(tottime, N, param, inten, timeleft):
    Y = random.random()
    def f(X):
        # Integrate sum of intensities from 0 to X
        integral, _ = quad(lambda x: sum(inten(tottime + x, N, param)), 0, X, limit=200)
        return 1 - math.exp(-integral) - Y
    try:
        sol = root_scalar(f, bracket=[0, timeleft], method='bisect', xtol=1e-5)
        return sol.root
    except ValueError:
        # In case no root is found, return a value greater than timeleft
        return timeleft + 1

In [ ]:
def estradiol_to_transition_param(x, param):
    


In [ ]:
def inten(t, pops, params):
    # Intensity function and demographic functions combined into single func
    # Should return the values for birth*population, etc. as numbers not as rates

    # Current populations
    F, H, S, M = pops

    # Get the current estradiol level
    # e2_lvl = estradiol_lvl(t)

    # Find position in estrous cycle and use the appropriate transition
    # values for that point between stages
    tind = find_closest_timestamp(np.linspace(0,4,100)*24, t)

    # Given a cycle position, get the transition probabilities
    use_intensities = {}
    for key,val in transition_dict.items():
        use_intensities[key] = val[tind]

    # Ignore lambda values because these represent staibility not
    # a transition between spine classes
    deltaM = use_intensities['deltaM'][tind]
    gammaM2F = use_intensities['gammaM2F'][tind]
    gammaM2H = use_intensities['gammaM2H'][tind]
    gammaM2S = use_intensities['gammaM2S'][tind]
    # lambdaM = use_intensities['lambdaM'][tind]
    deltaS = use_intensities['deltaS'][tind]
    gammaS2F = use_intensities['gammaS2F'][tind]
    gammaS2H = use_intensities['gammaS2H'][tind]
    # lambdaS = use_intensities['lambdaS'][tind]
    gammaS2M = use_intensities['gammaS2M'][tind]
    deltaH = use_intensities['deltaH'][tind]
    gammaH2F = use_intensities['gammaH2F'][tind]
    # lambdaH = use_intensities['lambdaH'][tind]
    gammaH2S = use_intensities['gammaH2S'][tind]
    gammaH2M = use_intensities['gammaH2M'][tind]
    deltaF = use_intensities['deltaF'][tind]
    # lambdaF = use_intensities['lambdaF'][tind]
    gammaF2H = use_intensities['gammaF2H'][tind]
    gammaF2S = use_intensities['gammaF2S'][tind]
    gammaF2M = use_intensities['gammaF2M'][tind]
    betaF = use_intensities['betaF'][tind]
    betaH = use_intensities['betaH'][tind]
    betaS = use_intensities['betaS'][tind]
    betaM = use_intensities['betaM'][tind]

    dFdt = betaF - deltaF*F + gammaH2F*H + gammaS2F*S + gammaM2F*M - F*(gammaF2H + gammaF2S + gammaF2M)
    dHdt = betaH - deltaH*H + gammaF2H*F + gammaS2H*S + gammaM2H*M - H*(gammaH2F + gammaH2S + gammaH2M)
    dSdt = betaS - deltaS*S + gammaF2S*F + gammaH2S*H + gammaM2S*M - S*(gammaS2F + gammaS2H + gammaS2M)
    dMdt = betaM - deltaM*M + gammaF2M*F + gammaH2M*H + gammaS2M*S - M*(gammaM2F + gammaM2H + gammaM2S)

    
    
    # Intensities should be a (20,) 

    return intensities


0